<a href="https://colab.research.google.com/github/jashvidesai/ORF-Thesis/blob/main/ALNSWithRemovalOperator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First portion is the initial soln, copied from the previous algorithm file.

In [24]:
import numpy as np
import heapq
import random
from copy import deepcopy

# Problem Parameters
V = range(18)  # Nodes including depot
V_star = range(1, 18)  # Customer nodes
K = range(8)  # Vehicles
Q = [100, 120, 98, 105, 92, 92, 96, 94]  # Vehicle capacities
vehicle_speed = 60  # Speed in km/h
fixed_costs = [120, 138, 118, 122, 110, 110, 115, 113]  # Fixed costs per vehicle

np.random.seed(42)
distances = np.random.randint(10, 51, size=(len(V), len(V)))
for i in V:
    distances[i, i] = 0
    for j in range(i + 1, len(V)):
        distances[j, i] = distances[i, j]
for i in V:
    for j in V:
        for k in V:
            if distances[i, j] > distances[i, k] + distances[k, j]:
                distances[i, j] = distances[i, k] + distances[k, j]

t = (distances / vehicle_speed) * 60  # travel times in minutes
c = distances * 0.093  # travel costs (arbitrary scaling for fuel)

# Delivery and pickup demands
d = [0, 21, 48, 32, 34, 33, 10, 39, 20, 41, 28, 33, 35, 39, 37, 25, 36, 27]  # Delivery demands
p = [0, 37, 7, 22, 29, 35, 7, 28, 36, 26, 27, 6, 31, 6, 30, 21, 37, 13]  # Pickup demands

# Time windows and service times
a = [0, 15, 26, 20, 11, 14, 27, 7, 22, 29, 28, 17, 11, 29, 8, 17, 24, 5]  # Earliest arrival times
b = [300] * 18  # Latest departure times
s = [8, 8, 6, 5, 9, 9, 6, 4, 4, 9, 9, 8, 5, 3, 4, 9, 8, 7]  # Service times

# define a vehicle class to store critical information
class Vehicle:
    def __init__(self, vehicle_id, capacity, speed, cost, depot):
        self.vehicle_id = vehicle_id
        self.capacity = capacity
        self.speed = speed
        self.cost = cost
        self.route = [] # (customer_id, delivered, picked_up, arrival_time)
        self.current_time = 0

        # Updated Capacity Tracking
        self.full_vials = int(self.capacity * 0.75) # Start with 50% full vials
        self.empty_vials = 0 # Initially no empty vials
        self.empty_space = self.capacity - (self.full_vials + self.empty_vials) # Available space
        self.current_location = depot

    def can_add_customer(self, node, travel_time):
        """
        Check if the vehicle can arrive within the time window.
        Returns (boolean) feasibility flag and expected arrival time.
        """
        # check if the vehicle has capacity
        if self.empty_vials == self.capacity:
            return False, self.current_time

        arrival_time = self.current_time + travel_time
        if arrival_time > b[node]:  # Too late
            return False, arrival_time
        if arrival_time < a[node]:  # Arrive early, wait
            arrival_time = a[node]
        return True, arrival_time

    def add_customer(self, node, delivery_demand, pickup_demand, travel_time):
        """
        Assigns a customer to the vehicle while ensuring delivery before pickup
        and allowing split deliveries.
        """
        # Step 1: Check feasibility (time window constraints)
        feasible, arrival_time = self.can_add_customer(node, travel_time)
        if not feasible:
            return False, delivery_demand, pickup_demand

        # Step 2: Deliver first, and update the vehicle load values
        delivered = min(self.full_vials, delivery_demand)
        self.full_vials -= delivered
        delivery_demand -= delivered
        self.empty_space = self.capacity - (self.full_vials + self.empty_vials)

        # Step 3: Pick up second, and update the vehicle load values
        picked_up = min(self.empty_space, pickup_demand)
        self.empty_vials += picked_up
        pickup_demand -= picked_up
        self.empty_space = self.capacity - (self.full_vials + self.empty_vials)

        # Step 4: Save customer in route & update time
        self.route.append((node, delivered, picked_up, arrival_time))
        self.current_time = arrival_time + s[node]
        self.current_location = node

        # ** Debugging Print Statements **
        print(f"Vehicle {self.vehicle_id} visited Customer {node}:")
        print(f"   - Delivered {delivered} (Remaining at customer: {delivery_demand})")
        print(f"   - Picked Up {picked_up} (Remaining at customer: {pickup_demand})")
        print(f"   - Vehicle State: Full Vials = {self.full_vials}, Empty Vials = {self.empty_vials}, Empty Space = {self.empty_space}")


        return True, delivery_demand, pickup_demand

    def return_to_depot(self, depot, t, depot_service_time=30):
        """
        Sends the vehicle back to the depot when full of empty vials.
        Resets load and allows for redeployment.
        """
        travel_time_to_depot = t[self.current_location][depot] # Retrieve from travel time matrix
        self.route.append((depot, 0, 0, self.current_time + travel_time_to_depot)) # Add depot return
        self.current_time += travel_time_to_depot + depot_service_time # Add travel and depot service time

        # Reset vehicle load: Start with 75% full vials, empty vials reset to 0
        self.full_vials = int(self.capacity * 0.75)
        self.empty_vials = 0
        self.empty_space = self.capacity - (self.full_vials + self.empty_vials)

        print(f"Vehicle {self.vehicle_id} returned to depot at time {self.current_time} and reset.")

# visiting order is based on distance to depot
def compute_ordered_route(V_star, depot, distances):
    """
    Computes a heuristic visiting order:
    1. Start with the farthest customer from the depot.
    2. Sequentially add the nearest unvisited customer.
    """
    if not V_star:
        return []

    start_node = max(V_star, key=lambda c: distances[depot][c])  # Start with farthest
    ordered_route = [start_node]
    remaining_nodes = set(V_star) - {start_node}

    while remaining_nodes:
        last_node = ordered_route[-1]
        next_node = min(remaining_nodes, key=lambda c: distances[last_node][c])  # Nearest neighbor
        ordered_route.append(next_node)
        remaining_nodes.remove(next_node)

    return ordered_route

# compute a new ordered route based on proximity to depot (after depot service)
def compute_nearest_ordered_route(V_star, depot, distances):
    """
    Computes a heuristic visiting order:
    1. Start with the closest customer to the depot.
    2. Sequentially add the nearest unvisited customer.
    """
    if not V_star:
        return []

    start_node = min(V_star, key=lambda c: distances[depot][c])  # Start with nearest
    ordered_route = [start_node]
    remaining_nodes = set(V_star) - {start_node}

    while remaining_nodes:
        last_node = ordered_route[-1]
        next_node = min(remaining_nodes, key=lambda c: distances[last_node][c])  # Nearest neighbor
        ordered_route.append(next_node)
        remaining_nodes.remove(next_node)

    return ordered_route

def generate_sequential_solution(V, V_star, K, Q, d, p, a, b, s, t, depot, distances):
    """
    Deploys vehicles sequentially, prioritizing larger vehicles first.
    If a vehicle is redeployed, it follows a new nearest-neighbor order.
    """
    initial_ordered_route = compute_ordered_route(V_star, depot, distances)  # Initial farthest-first order

    # Deploy largest vehicles first
    vehicles = sorted(
        [Vehicle(k, Q[k], vehicle_speed, fixed_costs[k], depot) for k in K],
        key=lambda v: v.capacity,
        reverse=True
    )

    remaining_deliveries = {i: d[i] for i in V_star} # dictionary to track remaining delivery demand
    remaining_pickups = {i: p[i] for i in V_star} # dictionary to track remaining pickup demand

    for vehicle in vehicles:
        print(f"\nDeploying Vehicle {vehicle.vehicle_id} (Capacity {vehicle.capacity})\n")

        current_route = initial_ordered_route[:]

        while True:  # Keep redeploying the vehicle until it can’t serve any more customers
            all_customers_served = True  # Flag to check if all customers are served

            # skips if fully served
            for customer in current_route[:]:
                if remaining_deliveries[customer] == 0 and remaining_pickups[customer] == 0:
                    continue

                success, new_remaining_delivery, new_remaining_pickup = vehicle.add_customer(
                    customer, remaining_deliveries[customer], remaining_pickups[customer],
                    t[vehicle.current_location][customer]
                )

                if not success:
                    break

                remaining_deliveries[customer] = new_remaining_delivery
                remaining_pickups[customer] = new_remaining_pickup
                all_customers_served = False  # At least one customer was served

            vehicle.return_to_depot(depot, t, depot_service_time=30)  # 30-min depot service time

            # Check if there are still unfulfilled customers
            if all_customers_served:
                break  # No point in redeploying, so exit

            # Compute a new order for redeployment based on nearest-first heuristic
            unserved_customers = [i for i in V_star if remaining_deliveries[i] > 0 or remaining_pickups[i] > 0]
            current_route = compute_nearest_ordered_route(unserved_customers, depot, distances)

    return vehicles, remaining_deliveries, remaining_pickups

# Run the updated solution
vehicles_updated, remaining_deliveries, remaining_pickups = generate_sequential_solution(
    V, V_star, K, Q, d, p, a, b, s, t, depot=0, distances=distances
)

print("\n**Final Vehicle Routes and Loads**")
for vehicle in vehicles_updated:
    print(f"Vehicle {vehicle.vehicle_id}: Route {vehicle.route}, Final Load: Full={vehicle.full_vials}, Empty={vehicle.empty_vials}")

print("\n**Final Remaining Deliveries**")
for customer, remaining in remaining_deliveries.items():
    if remaining > 0:
        print(f"Customer {customer}: {remaining} units left to deliver")

print("\n**Final Remaining Pickups**")
for customer, remaining in remaining_pickups.items():
    if remaining > 0:
        print(f"Customer {customer}: {remaining} units left to pick up")


Deploying Vehicle 1 (Capacity 120)

Vehicle 1 visited Customer 5:
   - Delivered 33 (Remaining at customer: 0)
   - Picked Up 35 (Remaining at customer: 0)
   - Vehicle State: Full Vials = 57, Empty Vials = 35, Empty Space = 28
Vehicle 1 visited Customer 6:
   - Delivered 10 (Remaining at customer: 0)
   - Picked Up 7 (Remaining at customer: 0)
   - Vehicle State: Full Vials = 47, Empty Vials = 42, Empty Space = 31
Vehicle 1 visited Customer 4:
   - Delivered 34 (Remaining at customer: 0)
   - Picked Up 29 (Remaining at customer: 0)
   - Vehicle State: Full Vials = 13, Empty Vials = 71, Empty Space = 36
Vehicle 1 visited Customer 11:
   - Delivered 13 (Remaining at customer: 20)
   - Picked Up 6 (Remaining at customer: 0)
   - Vehicle State: Full Vials = 0, Empty Vials = 77, Empty Space = 43
Vehicle 1 visited Customer 9:
   - Delivered 0 (Remaining at customer: 41)
   - Picked Up 26 (Remaining at customer: 0)
   - Vehicle State: Full Vials = 0, Empty Vials = 103, Empty Space = 17
Vehi

Removal Operators!

In [10]:
# Random Removal
def random_removal(solution, p, remaining_deliveries, remaining_pickups):
    modified_solution = deepcopy(solution)
    all_customers = set()

    # Extract all customers currently in the solution
    for vehicle in modified_solution:
        for stop in vehicle.route:
            if stop[0] != 0:  # Exclude depot
                all_customers.add(stop[0])

    num_to_remove = int(p * len(all_customers))
    customers_to_remove = random.sample(list(all_customers), min(num_to_remove, len(all_customers)))  # Avoid oversampling

    # Remove customers from their respective routes
    for vehicle in modified_solution:
        vehicle.route = [stop for stop in vehicle.route if stop[0] not in customers_to_remove]

    # Set remaining demand of removed customers to 0 (Prevents reallocation)
    for customer in customers_to_remove:
        remaining_deliveries[customer] = 0
        remaining_pickups[customer] = 0

    return modified_solution, customers_to_remove, remaining_deliveries, remaining_pickups

# Example usage: Apply Random Removal on the generated initial solution
modified_random, removed_random, updated_deliveries, updated_pickups = random_removal(
    vehicles_updated, 0.5, remaining_deliveries, remaining_pickups
)
print("\nRemoved Customers (Random Removal):", removed_random)
print("Updated Remaining Deliveries:", updated_deliveries)
print("Updated Remaining Pickups:", updated_pickups)


Removed Customers (Random Removal): [14, 10, 16, 1, 2, 17, 12, 15]
Updated Remaining Deliveries: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0}
Updated Remaining Pickups: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0}


In [22]:
# Related Removal
def related_removal(solution, p, distances, remaining_deliveries, remaining_pickups):
    modified_solution = deepcopy(solution)  # Creates a copy of the solution
    all_customers = set()

    # Extract all customers currently in the solution
    for vehicle in modified_solution:
        for stop in vehicle.route:
            if stop[0] != 0:  # Exclude depot
                all_customers.add(stop[0])

    if not all_customers:
        return modified_solution, [], remaining_deliveries, remaining_pickups

    num_to_remove = int(p * len(all_customers))
    seed_customer = random.choice(list(all_customers))  # Randomly select a seed customer

    # Compute distances from the seed customer to all others and sort
    related_customers = sorted(all_customers, key=lambda c: distances[seed_customer][c])
    customers_to_remove = related_customers[:min(num_to_remove, len(related_customers))]  # Avoid oversampling

    # Remove selected customers from all routes
    for vehicle in modified_solution:
        vehicle.route = [stop for stop in vehicle.route if stop[0] not in customers_to_remove]

    # Set remaining demand of removed customers to 0 (Prevents reallocation)
    for customer in customers_to_remove:
        remaining_deliveries[customer] = 0
        remaining_pickups[customer] = 0

    return modified_solution, customers_to_remove, remaining_deliveries, remaining_pickups

# Example Usage
modified_related, removed_related, updated_deliveries, updated_pickups = related_removal(
    vehicles_updated, 0.5, distances, remaining_deliveries, remaining_pickups
)
print("\nRemoved Customers (Related Removal):", removed_related)
print("Updated Remaining Deliveries:", updated_deliveries)
print("Updated Remaining Pickups:", updated_pickups)


Removed Customers (Related Removal): [14, 11, 17, 1, 4, 9, 16, 2]
Updated Remaining Deliveries: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0}
Updated Remaining Pickups: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0}


In [28]:
# Worst Removal
def worst_removal(solution, p, distances, fixed_costs, remaining_deliveries, remaining_pickups):
    modified_solution = deepcopy(solution)
    all_customers = set()
    customer_total_cost = {}

    # Extract all customers and calculate cost impact
    for vehicle in modified_solution:
        for i, stop in enumerate(vehicle.route):
            if stop[0] != 0:  # Exclude depot
                all_customers.add(stop[0])

                # Get previous and next stops
                prev_stop = vehicle.route[i - 1] if i > 0 else (0, 0, 0, 0)
                next_stop = vehicle.route[i + 1] if i < len(vehicle.route) - 1 else (0, 0, 0, 0)  # Depot if last stop

                # Compute cost impact: added travel distance and vehicle fixed cost
                cost = (
                    distances[prev_stop[0]][stop[0]] +
                    distances[stop[0]][next_stop[0]] -
                    distances[prev_stop[0]][next_stop[0]] +
                    fixed_costs[vehicle.vehicle_id]
                )

                # Aggregate total cost contribution per customer
                customer_total_cost[stop[0]] = customer_total_cost.get(stop[0], 0) + cost

    if not all_customers:
        return modified_solution, [], remaining_deliveries, remaining_pickups

    num_to_remove = int(p * len(all_customers))

    # Sort customers by total cost contribution in descending order
    sorted_customers = sorted(customer_total_cost.items(), key=lambda x: x[1], reverse=True)

    # Extract the highest-cost customers for removal
    customers_to_remove = [customer for customer, _ in sorted_customers[:min(num_to_remove, len(sorted_customers))]]

    # Remove selected customers from all routes
    for vehicle in modified_solution:
        vehicle.route = [stop for stop in vehicle.route if stop[0] not in customers_to_remove]

    # Set remaining demand of removed customers to 0 (Prevents reallocation)
    for customer in customers_to_remove:
        remaining_deliveries[customer] = 0
        remaining_pickups[customer] = 0

    return modified_solution, customers_to_remove, remaining_deliveries, remaining_pickups

# Example Usage
modified_worst, removed_worst, updated_deliveries, updated_pickups = worst_removal(
    vehicles_updated, 0.5, distances, fixed_costs, remaining_deliveries, remaining_pickups
)

print("\nRemoved Customers (Worst Removal):", removed_worst)
print("Updated Remaining Deliveries:", updated_deliveries)
print("Updated Remaining Pickups:", updated_pickups)


Removed Customers (Worst Removal): [10, 15, 17, 3, 1, 2, 8, 14]
Updated Remaining Deliveries: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0}
Updated Remaining Pickups: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0}
